# Problem 1

Using an element `as-is`. This assumes the following:

- All dependent element packages are installed
- All tables are necessary and shouldn't be altered
- All element code doesn't require user specific functions in their make methods


In [1]:
import datajoint as dj

dj.Schema("my_custom_workflow_child").drop(force=True)
dj.Schema("my_custom_workflow_parent").drop(force=True)


Connecting root@localhost:3306


The database prefix is determined by `dj_local_conf.json` ('database.prefix' key) at the working directory.

This wil set the custom database name for all imported elements.

In [2]:
dj.config

{   'connection.charset': '',
    'connection.init_function': None,
    'database.host': 'localhost',
    'database.password': 'simple',
    'database.port': 3306,
    'database.prefix': 'my_custom_workflow',
    'database.reconnect': True,
    'database.use_tls': False,
    'database.user': 'root',
    'display.limit': 12,
    'display.show_tuple_count': True,
    'display.width': 14,
    'enable_python_native_blobs': True,
    'fetch_format': 'array',
    'loglevel': 'WARNING',
    'safemode': True}

Import the pipeline elements from:

- `src/datajoint_extension/pipeline/parent.py`
- `src/datajoint_extension/pipeline/child.py`

All necessary tables already activated if imported from `.tables`

In [3]:
from datajoint_extension.tables import Child, Parent, Sibling
dj.list_schemas()


['my_custom_workflow_child',
 'my_custom_workflow_parent',
 'mysql',
 'performance_schema',
 'sys']

Add manual entries, the populate downstream content.

In [4]:
Parent.insert1({"field": "parent"})
Sibling.insert1({"field2": "sibling"})
Child.populate()


This is the default output if not editing anything.

In [5]:
Child()


field,field2,path
parent,sibling,/Users/joseph/Documents/projects/datajoint-lazy-schema/src/datajoint_extension


### Content from `parent.py`

In [ ]:
import datajoint as dj
from datajoint_extension.utils.schema import LazySchema

# Argument to LazySchema is optional. If it's empty and:
# if "workflow" is defined in dj.config, use that as the database prefix,
# otherwise use the package import name as the prefix.
# the module name ("parent") will be appended to the prefix, sep='_'.
schema = LazySchema()


@schema
class Parent(dj.Manual):
    definition = """
    field : VARCHAR(8)
    """


### Content from `child.py`

In [ ]:
from functools import singledispatch

import datajoint as dj
from datajoint_extension.pipeline.parent import Parent
from datajoint_extension.utils.generic import pkg_abspath
from datajoint_extension.utils.schema import LazySchema

schema = LazySchema()


@singledispatch
def some_schema_requirement(obj):
    return pkg_abspath(obj)


@schema
class Sibling(dj.Manual):
    definition = """
    field2 : VARCHAR(8)
    """


@schema
class Child(dj.Computed):
    definition = """
    -> Parent
    -> Sibling
    path : VARCHAR(512)
    """

    def make(self, key):
        key |= {"path": some_schema_requirement(None)}
        self.insert1(key)
